In [1]:
from utils.load_googlesheet import *

df = load_google_sheets_data("이탈_RAW")
df.head()

2025-10-01 14:56:28.301 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-10-01 14:56:28.303 WARNING streamlit.runtime.caching.cache_data_api: No runtime found, using MemoryCacheStorageManager
2025-10-01 14:56:28.303 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-01 14:56:28.387 
  command:

    streamlit run /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-10-01 14:56:28.387 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-01 14:56:28.388 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-01 14:56:28.388 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


[2025-10-01 14:56:28.388871] Google Sheet 데이터 로드


2025-10-01 14:56:28.894 Thread 'Thread-4': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-01 14:56:28.895 Thread 'Thread-4': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-01 14:56:28.895 Thread 'Thread-4': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-01 14:56:42.973 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-01 14:56:42.974 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-01 14:56:42.974 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


[2025-10-01 14:56:42.802168] 데이터 로드 성공: 37835행, 35열


,단계,payment_regdate,이탈여부,user_No,lvt,과외상태,stage_count,cycle_count,수업상태,과외일정,...,email_id,done_month,교과/탐구,학년,option,중단예정일,중단 예정 DONEMONTH,최초 개월 수,done_count,glvt_no
0,9,2024-01-01 10:37,P,612220,72405,FINISH,10,8,RESERVATION,2025-03-03 21:00:00,...,daeun01@naver.com,15.125,교과,중3,W2_H60,2025-02-28 12:10,15.125,6,121,
1,9,2024-01-01 13:17,P,620252,78506,FINISH,4,2,RESERVATION,2024-11-15 22:30:00,...,won95580@naver.com,9.7493,교과,N수생,W2_H60,2024-07-08 10:11,9.7493,6,85,
2,9,2024-01-01 13:17,P,620252,78508,FINISH,4,8,RESERVATION,2024-09-01 13:00:00,...,won95580@naver.com,6.0396,교과,N수생,W2_H90,2024-10-07 10:59,6.0396,3,70,
3,8,2024-01-01 13:38,P,620153,78462,FINISH,1,16,RESERVATION,2024-05-25 10:00:00,...,victoryzoo1231@naver.com,3.75,교과,고3,W1_H90,2024-05-23 14:46,3.75,6,15,
4,9,2024-01-01 14:07,P,608191,78509,AUTO_FINISH,15,4,DONE,2025-04-29 20:30:00,...,psw492@gmail.com,17.5,교과,중2,W1_H60,2025-04-28 13:31,17.75,3,71,


In [2]:
df_processed = processing_google_sheet(df)
df_processed.head()

[2025-10-01 14:56:42.987844] 전처리 시작
[2025-10-01 14:56:43.282464] 전처리 완료: shape=(37825, 18)


,결제등록일,lvt,user_No,option,단계,이탈여부,donemonth_raw,donemonth,duration_days,학년,교과/탐구,결제개월수,stage_count,cycle_count,과외상태,수업상태,중단예정일,중단 예정 DONEMONTH
0,2024-01-01 10:37:00,72405,612220,W2_H60,9,1,15.1250,15.1250,427,중3,교과,6,10,8,FINISH,RESERVATION,2025-02-28 12:10:00,15.1250
1,2024-01-01 13:17:00,78506,620252,W2_H60,9,1,9.7493,9.7493,273,N수생,교과,6,4,2,FINISH,RESERVATION,2024-07-08 10:11:00,9.7493
2,2024-01-01 13:17:00,78508,620252,W2_H90,9,1,6.0396,6.0396,175,N수생,교과,3,4,8,FINISH,RESERVATION,2024-10-07 10:59:00,6.0396
3,2024-01-01 13:38:00,78462,620153,W1_H90,8,1,3.7500,3.7500,105,고3,교과,6,1,16,FINISH,RESERVATION,2024-05-23 14:46:00,3.7500
4,2024-01-01 14:07:00,78509,608191,W1_H60,9,1,17.5000,17.5000,490,중2,교과,3,15,4,AUTO_FINISH,DONE,2025-04-28 13:31:00,17.7500


In [6]:
df_count = df_processed.groupby(pd.Grouper(key="결제등록일", freq="ME")).agg(
    신규수업수=("lvt", "count")
).reset_index()

df_chrun_count = df_processed[df_processed['이탈여부'] == 1].pivot_table(
    index=pd.Grouper(key="결제등록일", freq="ME"),
    columns='단계',
    values='lvt',
    aggfunc='count',
    fill_value=0
)
df_chrun_count.columns = [f'단계{i}_이탈수' for i in df_chrun_count.columns]
df_chrun_count = df_chrun_count.reset_index()

df_merged = pd.merge(df_count, df_chrun_count, how = 'inner', on = '결제등록일')
df_merged.head()

,결제등록일,신규수업수,단계1_이탈수,단계2_이탈수,단계3_이탈수,단계4_이탈수,단계5_이탈수,단계6_이탈수,단계7_이탈수,단계8_이탈수,단계9_이탈수
0,2024-01-31,2306,109,17,76,102,51,3,34,293,1461
1,2024-02-29,1461,43,20,49,84,38,0,30,191,896
2,2024-03-31,1889,79,20,56,145,8,5,36,313,1082
3,2024-04-30,1354,62,16,55,101,7,4,26,222,756
4,2024-05-31,2217,90,26,89,192,1,2,44,347,1221


In [19]:
result_df = pd.DataFrame()
result_df['연-월'] = df_merged['결제등록일']
result_df['결제'] = (df_merged['단계1_이탈수'] / df_merged['신규수업수'] * 100)
result_df['과외신청서'] = (df_merged['단계2_이탈수'] / df_merged['신규수업수'] * 100)
result_df['결제직후매칭전'] = result_df['결제'] + result_df['과외신청서']
result_df['매칭직후첫수업전'] = (df_merged['단계3_이탈수']/df_merged['신규수업수'] * 100)

In [20]:
def calculate_churn_rate_by_donemonth(df_processed, start_date, end_date, min_donemonth, max_donemonth, total_count, second_cycle=True):
    """
    특정 기간(start_date ~ end_date) 동안,
    donemonth 범위(min_donemonth < x <= max_donemonth),
    단계=2, 이탈여부=1 조건을 만족하는 비율을 계산
    """
    if second_cycle:
        filtered_df = df_processed[
            (df_processed['결제등록일'] >= start_date) &
            (df_processed['결제등록일'] < end_date) &
            (df_processed['donemonth_raw'] > min_donemonth) &
            (df_processed['donemonth_raw'] <= max_donemonth) &
            (df_processed['cycle_count'] == 2) &
            (df_processed['이탈여부'] == 1)   # 엑셀의 "P" → 파이썬 전처리에서 int 1
        ]
    else:
        filtered_df = df_processed[
            (df_processed['결제등록일'] >= start_date) &
            (df_processed['결제등록일'] < end_date) &
            (df_processed['donemonth_raw'] > min_donemonth) &
            (df_processed['donemonth_raw'] <= max_donemonth) &
            (df_processed['이탈여부'] == 1)   # 엑셀의 "P" → 파이썬 전처리에서 int 1
        ]
        
    if total_count > 0:
        return len(filtered_df) / total_count * 100
    else:
        return 0.0

In [21]:
result_df['첫수업후2회차수업전'] = 0.0

for i, row in df_merged.iterrows():
    month_start = row['결제등록일'].replace(day=1)             # 해당 월 시작일
    next_month = month_start + pd.DateOffset(months=1)         # 다음 달 시작일

    churn_rate = calculate_churn_rate_by_donemonth(
        df_processed,
        month_start,
        next_month,
        0,       # donemonth > 0
        0.25,    # donemonth <= 0.25
        row['신규수업수']   # 엑셀의 분모 G4
    )

    result_df.loc[i, '첫수업후2회차수업전'] = churn_rate

result_df['첫수업후DM1이하'] = 0.0

for i, row in df_merged.iterrows():
    month_start = row['결제등록일'].replace(day=1)             # 해당 월 시작일
    next_month = month_start + pd.DateOffset(months=1)         # 다음 달 시작일

    churn_rate = calculate_churn_rate_by_donemonth(
        df_processed,
        month_start,
        next_month,
        0.0,     
        1.0,  
        row['신규수업수'],   # 엑셀의 분모 G4
        second_cycle = False
    )

    result_df.loc[i, '첫수업후DM1이하'] = churn_rate

In [23]:
result_df['2회차수업후DM1이하'] = result_df['첫수업후DM1이하'] - result_df['첫수업후2회차수업전']
result_df['매칭직후DM1이하'] = result_df['매칭직후첫수업전'] + result_df['첫수업후2회차수업전'] + result_df['2회차수업후DM1이하']

In [24]:
result_df.head()

,연-월,결제,과외신청서,결제직후매칭전,매칭직후첫수업전,첫수업후2회차수업전,첫수업후DM1이하,2회차수업후DM1이하,매칭직후DM1이하
0,2024-01-31,4.7268,0.737207,5.464007,3.29575,3.078925,8.846487,5.767563,12.142238
1,2024-02-29,2.94319,1.368925,4.312115,3.353867,3.832991,11.225188,7.392197,14.579055
2,2024-03-31,4.182107,1.058761,5.240868,2.964531,3.546850,11.011117,7.464267,13.975648
3,2024-04-30,4.579025,1.181684,5.760709,4.062038,3.249631,10.856721,7.607090,14.918759
4,2024-05-31,4.05954,1.172756,5.232296,4.014434,3.879116,11.727560,7.848444,15.741994


In [31]:
result_df.mean()

연-월            2024-11-29 16:00:00
결제                        4.939496
과외신청서                     1.127411
결제직후매칭전                   6.066907
매칭직후첫수업전                  3.755174
첫수업후2회차수업전                3.696406
첫수업후DM1이하                11.395172
2회차수업후DM1이하               7.698765
매칭직후DM1이하                15.150346
dtype: object